In [101]:
# https://www.kaggle.com/datasets/pavansanagapati/images-dataset/data

In [1]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.preprocessing as preprocessing
import keras
import keras.layers as layers

In [2]:
device = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(device[0], True)

In [3]:
input_shape = (512, 512, 3)
data = tf.keras.preprocessing.image_dataset_from_directory(
    "../../../resources/image/data",
    image_size=(512, 512)
)

Found 3606 files belonging to 8 classes.


In [4]:
num_classes = 8
images = []
labels = []

for image_batch, label_batch in data:
    images.append(image_batch.numpy())
    labels.append(label_batch.numpy())

all_images = np.concatenate(images)
all_labels = np.concatenate(labels)

In [5]:
all_images.shape, all_labels.shape

((3606, 512, 512, 3), (3606,))

# Image processing

In [6]:
x_train, x_test, y_train, y_test = train_test_split(all_images, all_labels, random_state=42, test_size=0.3)

In [7]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [8]:
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
x_train.shape, x_test.shape

((2524, 512, 512, 3), (1082, 512, 512, 3))

In [9]:
x_val = x_train[:300]
partial_x_train = x_train[300:]

y_val = y_train[:300]
partial_y_train = y_train[300:]

# Model

In [10]:
model = keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3, 3), input_shape=input_shape))
#model.add(layers.MaxPool2D(pool_size=(2, 2)))
model.add(layers.Conv2D(32, kernel_size=(3, 3)))
model.add(layers.MaxPool2D(pool_size=(2, 2)))
model.add(layers.Conv2D(32, kernel_size=(3, 3)))
model.add(layers.MaxPool2D(pool_size=(2, 2)))
model.add(layers.Conv2D(23, kernel_size=(3, 3)))
model.add(layers.MaxPool2D(pool_size=(2, 2)))
model.add(layers.Conv2D(32, kernel_size=(3, 3)))
model.add(layers.MaxPool2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(num_classes, activation="softmax"))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 510, 510, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 508, 508, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 254, 254, 32)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 252, 252, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 126, 126, 32)     0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 124, 124, 23)      6

In [113]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["acc"]
)

In [114]:
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=20,
    validation_data=(x_val, y_val),
    batch_size=32
)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
model.evaluate(x_test, y_test)

# Display performance curves

In [ ]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, 21)

plt.plot(epochs, loss_values, 'bo', label="Training")
plt.plot(epochs, val_loss_values, 'b', label="Validation")

plt.title("Loss during training process")
plt.xlabel("Nb epochs")
plt.ylabel("Loss")
plt.legend()

In [ ]:
history_dict = history.history
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

epochs = range(1, 21)

plt.plot(epochs, acc_values, 'bo', label="Training")
plt.plot(epochs, val_acc_values, 'b', label="Validation")

plt.title("Accuracy during training process")
plt.xlabel("Nb epochs")
plt.ylabel("Accuracy")
plt.legend()